In [3]:
from __future__ import division, print_function
import os
import sys
from datetime import datetime, timedelta


import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt

from scipy import stats
%matplotlib inline

from pyspark.sql import SparkSession

NUM_PARTITIONS = 1024
spark = ( SparkSession
             .builder
             .config("spark.app.name","BootstrappingConfig")
             .config("spark.yarn.queue","root.sp_analytics.spark")
             .config("spark.default.serializer","org.apache.spark.serializer.KryoSerializer")
             .config("spark.speculation","true")
             .config("spark.sql.autoBroadcastJoinThreshold","-1")
             .config("spark.default.parallelism",str(NUM_PARTITIONS))
             .config("spark.sql.shuffle.partitions",str(NUM_PARTITIONS))
             .config("spark.yarn.executor.memoryOverhead", str(8192))
             .enableHiveSupport()
             .getOrCreate() )

sc = spark.sparkContext

In [11]:
# put the table name of the bootstrapping result here
account = "account_4"
user_name = "lijia" # put in whichever user name, your own , or spmeta
#test = spark.sql("show tables in {0} like '{1}_bootstrapping_*'".format(user_name, account))
table_name = test.orderBy(test.tableName.desc()).first().tableName
#print(table_name)
table_name = "account_4_bootstrapping_20190228_nsalt_10000"
res = spark.table("{0}.{1}".format(user_name,table_name)).cache() 
metric_names = ['nits_bookings', 'nits_profit']

In [22]:
# first generate the descriptives statistics of the empirical distribution 
num_days   = [7, 14, 21, 28]
quantile_table = pd.DataFrame()
for num_day in num_days:
    
    subdata = res.filter(res.num_days == num_day).filter("paired_groups = '0_1'").toPandas()
    # manually filter out. 
    #subdata = subdata[np.abs(subdata.rdiff_gross_profit)<5]
    # get the 'true' mean of the underlying distribution, assuming it is the true after resampling many times 
    global_mean = ( subdata.groupby(["num_groups"],as_index=False)
                                                .agg({'rdiff_' + m:"mean" for m in metric_names}) )
    global_mean.columns = ['num_groups'] + metric_names
    subdata = ( subdata.merge(global_mean,on="num_groups",how="inner") )
    # empirical bootstrap step here: 
    for m in metric_names:
        subdata["eb_"+m] = subdata['rdiff_' + m] - subdata[m] 
    quantile_table = quantile_table.append( subdata.groupby(["num_days", "num_groups"],as_index=False)
                                           .quantile(q = [0.025, 0.975]).round(3)
                                           [['num_groups', "num_days"] + ['eb_' + m for m in metric_names]]
                                          )


quantile_table.index      = quantile_table.index.set_names('CI', level = 1)
quantile_table            = quantile_table.reset_index(level = 1)
quantile_table.CI         = (quantile_table.CI * 100).astype(str) + '%'
quantile_table.num_days   = quantile_table.num_days.astype(int)
quantile_table.num_groups = quantile_table.num_groups.astype(int)
quantile_table.columns = ["Confidence Interval", "Number of Groups", "Number of Days", 
                          "Random Salt {bookings}".format(bookings=metric_names[0]),
                          "Random Salt {profit}".format(profit=metric_names[1])]

In [23]:
random_salt_quantile_table = quantile_table
random_salt_quantile_table

,Confidence Interval,Number of Groups,Number of Days,Random Salt nits_bookings,Random Salt nits_profit
0,2.5%,2,7,-0.025,-0.021
0,97.5%,2,7,0.025,0.021
1,2.5%,4,7,-0.034,-0.032
1,97.5%,4,7,0.037,0.030
2,2.5%,10,7,-0.055,-0.048
2,97.5%,10,7,0.060,0.047
3,2.5%,20,7,-0.075,-0.068
3,97.5%,20,7,0.086,0.067
0,2.5%,2,14,-0.022,-0.020
0,97.5%,2,14,0.023,0.020


### compare with the smart salt group

In [47]:
smart_salt_quantile_table = pd.DataFrame()
number_of_salts = 10
for group_size in ("5percent","10percent", "25percent", "50percent"):
    
    table_name = 'account_4_smart_salt_bootstrapping_20190228_{0}_salt_numbers_{1}'.format(group_size, number_of_salts)
    print(table_name)
    smart_salt_bootstrapping = spark.sql("select * from lijia.{table_name}".format(table_name = table_name))\
                .filter("paired_groups == '0_1'")
    
    num_days   = [7, 14, 21, 28]
    quantile_table = pd.DataFrame()
    
    for num_day in num_days:
    
        subdata = smart_salt_bootstrapping.filter(smart_salt_bootstrapping.num_days == num_day).toPandas()
        # manually filter out. 
        #subdata = subdata[np.abs(subdata.rdiff_gross_profit)<5]
        # get the 'true' mean of the underlying distribution, assuming it is the true after resampling many times 
        global_mean = ( subdata.groupby(["num_groups"],as_index=False)
                                                    .agg({'rdiff_' + m:"mean" for m in metric_names}) )
        global_mean.columns = ['num_groups'] + metric_names
        subdata = ( subdata.merge(global_mean,on="num_groups",how="inner") )
        # empirical bootstrap step here: 
        for m in metric_names:
            subdata["eb_"+m] = subdata['rdiff_' + m] - subdata[m] 
        quantile_table = quantile_table.append( subdata.groupby(["num_days", "num_groups"],as_index=False)
                                               .quantile(q = [0.025, 0.975]).round(3)
                                               [['num_groups', "num_days"] + ['eb_' + m for m in metric_names]]
                                              )
    
    quantile_table.index      = quantile_table.index.set_names('CI', level = 1)
    quantile_table            = quantile_table.reset_index(level = 1)
    quantile_table.CI         = (quantile_table.CI * 100).astype(str) + '%'
    quantile_table.num_days   = quantile_table.num_days.astype(int)
    quantile_table.num_groups = quantile_table.num_groups.astype(int)
    quantile_table.columns = ["Confidence Interval", "Number of Groups", "Number of Days", 
                              "top 10 Smart Salt v1 {bookings}".format(bookings=metric_names[0]),
                              "top 10 Smart Salt v1 {profit}".format(profit=metric_names[1])]
    
    smart_salt_quantile_table = smart_salt_quantile_table.append(quantile_table)

smart_random_salt_comparison = random_salt_quantile_table.merge(smart_salt_quantile_table, how = 'inner')

account_4_smart_salt_bootstrapping_20190228_5percent_salt_numbers_10
account_4_smart_salt_bootstrapping_20190228_10percent_salt_numbers_10
account_4_smart_salt_bootstrapping_20190228_25percent_salt_numbers_10
account_4_smart_salt_bootstrapping_20190228_50percent_salt_numbers_10


In [30]:
smart_random_salt_comparison

,Confidence Interval,Number of Groups,Number of Days,Random Salt nits_bookings,Random Salt nits_profit,top 10 Smart Salt v1 nits_bookings,top 10 Smart Salt v1 nits_profit
0,2.5%,2,7,-0.025,-0.021,-0.011,-0.010
1,97.5%,2,7,0.025,0.021,0.012,0.008
2,2.5%,4,7,-0.034,-0.032,-0.029,-0.025
3,97.5%,4,7,0.037,0.030,0.036,0.018
4,2.5%,10,7,-0.055,-0.048,-0.035,-0.014
5,97.5%,10,7,0.060,0.047,0.028,0.021
6,2.5%,20,7,-0.075,-0.068,-0.041,-0.044
7,97.5%,20,7,0.086,0.067,0.063,0.009
8,2.5%,2,14,-0.022,-0.020,-0.014,-0.011
9,97.5%,2,14,0.023,0.020,0.012,0.008


In [48]:
smart_salt_quantile_table = pd.DataFrame()
number_of_salts = 100
for group_size in ("5percent","10percent", "25percent", "50percent"):
    
    table_name = 'account_4_smart_salt_bootstrapping_20190228_{0}_salt_numbers_{1}'.format(group_size, number_of_salts)
    print(table_name)
    smart_salt_bootstrapping = spark.sql("select * from lijia.{table_name}".format(table_name = table_name))\
                .filter("paired_groups == '0_1'")
    
    num_days   = [7, 14, 21, 28]
    quantile_table = pd.DataFrame()
    
    for num_day in num_days:
    
        subdata = smart_salt_bootstrapping.filter(smart_salt_bootstrapping.num_days == num_day).toPandas()
        # manually filter out. 
        #subdata = subdata[np.abs(subdata.rdiff_gross_profit)<5]
        # get the 'true' mean of the underlying distribution, assuming it is the true after resampling many times 
        global_mean = ( subdata.groupby(["num_groups"],as_index=False)
                                                    .agg({'rdiff_' + m:"mean" for m in metric_names}) )
        global_mean.columns = ['num_groups'] + metric_names
        subdata = ( subdata.merge(global_mean,on="num_groups",how="inner") )
        # empirical bootstrap step here: 
        for m in metric_names:
            subdata["eb_"+m] = subdata['rdiff_' + m] - subdata[m] 
        quantile_table = quantile_table.append( subdata.groupby(["num_days", "num_groups"],as_index=False)
                                               .quantile(q = [0.025, 0.975]).round(3)
                                               [['num_groups', "num_days"] + ['eb_' + m for m in metric_names]]
                                              )
    
    quantile_table.index      = quantile_table.index.set_names('CI', level = 1)
    quantile_table            = quantile_table.reset_index(level = 1)
    quantile_table.CI         = (quantile_table.CI * 100).astype(str) + '%'
    quantile_table.num_days   = quantile_table.num_days.astype(int)
    quantile_table.num_groups = quantile_table.num_groups.astype(int)
    quantile_table.columns = ["Confidence Interval", "Number of Groups", "Number of Days", 
                              "Top 100 Smart Salt v1 {bookings}".format(bookings=metric_names[0]),
                              "Top 100 Smart Salt v1 {profit}".format(profit=metric_names[1])]
    
    smart_salt_quantile_table = smart_salt_quantile_table.append(quantile_table)


account_4_smart_salt_bootstrapping_20190228_5percent_salt_numbers_100
account_4_smart_salt_bootstrapping_20190228_10percent_salt_numbers_100
account_4_smart_salt_bootstrapping_20190228_25percent_salt_numbers_100
account_4_smart_salt_bootstrapping_20190228_50percent_salt_numbers_100


In [49]:
smart_random_salt_comparison = smart_random_salt_comparison.merge(smart_salt_quantile_table, 
                                                                                how = 'inner')
smart_random_salt_comparison

,Confidence Interval,Number of Groups,Number of Days,Random Salt nits_bookings,Random Salt nits_profit,top 10 Smart Salt v1 nits_bookings,top 10 Smart Salt v1 nits_profit,Top 100 Smart Salt v1 nits_bookings,Top 100 Smart Salt v1 nits_profit
0,2.5%,2,7,-0.025,-0.021,-0.011,-0.010,-0.019,-0.015
1,97.5%,2,7,0.025,0.021,0.012,0.008,0.023,0.016
2,2.5%,4,7,-0.034,-0.032,-0.029,-0.025,-0.027,-0.019
3,97.5%,4,7,0.037,0.030,0.036,0.018,0.026,0.019
4,2.5%,10,7,-0.055,-0.048,-0.035,-0.014,-0.033,-0.034
5,97.5%,10,7,0.060,0.047,0.028,0.021,0.055,0.037
6,2.5%,20,7,-0.075,-0.068,-0.041,-0.044,-0.055,-0.058
7,97.5%,20,7,0.086,0.067,0.063,0.009,0.069,0.047
8,2.5%,2,14,-0.022,-0.020,-0.014,-0.011,-0.014,-0.015
9,97.5%,2,14,0.023,0.020,0.012,0.008,0.018,0.012


In [50]:
smart_salt_quantile_table = pd.DataFrame()
number_of_salts = 10
version = 'v3'
for group_size in ("5percent","10percent", "25percent", "50percent"):
    
    table_name = 'account_4_smart_salt_bootstrapping_{version}_20190228_{group_size}_salt_numbers_{number_of_salts}'\
                    .format(version = version, group_size = group_size, number_of_salts = number_of_salts)
    print(table_name)
    smart_salt_bootstrapping = spark.sql("select * from lijia.{table_name}".format(table_name = table_name))\
                .filter("paired_groups == '0_1'")
    
    num_days   = [7, 14, 21, 28]
    quantile_table = pd.DataFrame()
    
    for num_day in num_days:
    
        subdata = smart_salt_bootstrapping.filter(smart_salt_bootstrapping.num_days == num_day).toPandas()
        # manually filter out. 
        #subdata = subdata[np.abs(subdata.rdiff_gross_profit)<5]
        # get the 'true' mean of the underlying distribution, assuming it is the true after resampling many times 
        global_mean = ( subdata.groupby(["num_groups"],as_index=False)
                                                    .agg({'rdiff_' + m:"mean" for m in metric_names}) )
        global_mean.columns = ['num_groups'] + metric_names
        subdata = ( subdata.merge(global_mean,on="num_groups",how="inner") )
        # empirical bootstrap step here: 
        for m in metric_names:
            subdata["eb_"+m] = subdata['rdiff_' + m] - subdata[m] 
        quantile_table = quantile_table.append( subdata.groupby(["num_days", "num_groups"],as_index=False)
                                               .quantile(q = [0.025, 0.975]).round(3)
                                               [['num_groups', "num_days"] + ['eb_' + m for m in metric_names]]
                                              )
    
    quantile_table.index      = quantile_table.index.set_names('CI', level = 1)
    quantile_table            = quantile_table.reset_index(level = 1)
    quantile_table.CI         = (quantile_table.CI * 100).astype(str) + '%'
    quantile_table.num_days   = quantile_table.num_days.astype(int)
    quantile_table.num_groups = quantile_table.num_groups.astype(int)
    quantile_table.columns = ["Confidence Interval", "Number of Groups", "Number of Days", 
                              "Top 10 Smart Salt v3 {bookings}".format(bookings=metric_names[0]),
                              "Top 10 Smart Salt v3 {profit}".format(profit=metric_names[1])]
    
    smart_salt_quantile_table = smart_salt_quantile_table.append(quantile_table)


account_4_smart_salt_bootstrapping_v3_20190228_5percent_salt_numbers_10
account_4_smart_salt_bootstrapping_v3_20190228_10percent_salt_numbers_10
account_4_smart_salt_bootstrapping_v3_20190228_25percent_salt_numbers_10
account_4_smart_salt_bootstrapping_v3_20190228_50percent_salt_numbers_10


In [51]:
smart_random_salt_comparison = smart_random_salt_comparison.merge(smart_salt_quantile_table, how = 'inner')

In [52]:
smart_random_salt_comparison

,Confidence Interval,Number of Groups,Number of Days,Random Salt nits_bookings,Random Salt nits_profit,top 10 Smart Salt v1 nits_bookings,top 10 Smart Salt v1 nits_profit,Top 100 Smart Salt v1 nits_bookings,Top 100 Smart Salt v1 nits_profit,Top 10 Smart Salt v3 nits_bookings,Top 10 Smart Salt v3 nits_profit
0,2.5%,2,7,-0.025,-0.021,-0.011,-0.010,-0.019,-0.015,-0.012,-0.021
1,97.5%,2,7,0.025,0.021,0.012,0.008,0.023,0.016,0.021,-0.000
2,2.5%,4,7,-0.034,-0.032,-0.029,-0.025,-0.027,-0.019,-0.016,-0.022
3,97.5%,4,7,0.037,0.030,0.036,0.018,0.026,0.019,0.034,0.014
4,2.5%,10,7,-0.055,-0.048,-0.035,-0.014,-0.033,-0.034,-0.026,-0.036
5,97.5%,10,7,0.060,0.047,0.028,0.021,0.055,0.037,0.033,0.019
6,2.5%,20,7,-0.075,-0.068,-0.041,-0.044,-0.055,-0.058,-0.054,-0.035
7,97.5%,20,7,0.086,0.067,0.063,0.009,0.069,0.047,0.076,0.034
8,2.5%,2,14,-0.022,-0.020,-0.014,-0.011,-0.014,-0.015,-0.006,-0.017
9,97.5%,2,14,0.023,0.020,0.012,0.008,0.018,0.012,0.012,0.002


In [53]:
smart_salt_quantile_table = pd.DataFrame()
number_of_salts = 10
version = 'v2'
for group_size in ("5percent","10percent", "25percent", "50percent"):
    
    table_name = 'account_4_smart_salt_bootstrapping_{version}_20190228_{group_size}_salt_numbers_{number_of_salts}'\
                    .format(version = version, group_size = group_size, number_of_salts = number_of_salts)
    print(table_name)
    smart_salt_bootstrapping = spark.sql("select * from lijia.{table_name}".format(table_name = table_name))\
                .filter("paired_groups == '0_1'")
    
    num_days   = [7, 14, 21, 28]
    quantile_table = pd.DataFrame()
    
    for num_day in num_days:
    
        subdata = smart_salt_bootstrapping.filter(smart_salt_bootstrapping.num_days == num_day).toPandas()
        # manually filter out. 
        #subdata = subdata[np.abs(subdata.rdiff_gross_profit)<5]
        # get the 'true' mean of the underlying distribution, assuming it is the true after resampling many times 
        global_mean = ( subdata.groupby(["num_groups"],as_index=False)
                                                    .agg({'rdiff_' + m:"mean" for m in metric_names}) )
        global_mean.columns = ['num_groups'] + metric_names
        subdata = ( subdata.merge(global_mean,on="num_groups",how="inner") )
        # empirical bootstrap step here: 
        for m in metric_names:
            subdata["eb_"+m] = subdata['rdiff_' + m] - subdata[m] 
        quantile_table = quantile_table.append( subdata.groupby(["num_days", "num_groups"],as_index=False)
                                               .quantile(q = [0.025, 0.975]).round(3)
                                               [['num_groups', "num_days"] + ['eb_' + m for m in metric_names]]
                                              )
    
    quantile_table.index      = quantile_table.index.set_names('CI', level = 1)
    quantile_table            = quantile_table.reset_index(level = 1)
    quantile_table.CI         = (quantile_table.CI * 100).astype(str) + '%'
    quantile_table.num_days   = quantile_table.num_days.astype(int)
    quantile_table.num_groups = quantile_table.num_groups.astype(int)
    quantile_table.columns = ["Confidence Interval", "Number of Groups", "Number of Days", 
                              "Top 10 Smart Salt {version} {bookings}".format(version = version, bookings=metric_names[0]),
                              "Top 10 Smart Salt {version} {profit}".format(version = version, profit=metric_names[1])]
    
    smart_salt_quantile_table = smart_salt_quantile_table.append(quantile_table)


account_4_smart_salt_bootstrapping_v2_20190228_5percent_salt_numbers_10
account_4_smart_salt_bootstrapping_v2_20190228_10percent_salt_numbers_10
account_4_smart_salt_bootstrapping_v2_20190228_25percent_salt_numbers_10
account_4_smart_salt_bootstrapping_v2_20190228_50percent_salt_numbers_10


In [54]:
smart_random_salt_comparison = smart_random_salt_comparison.merge(smart_salt_quantile_table, how = 'inner')

In [64]:
column_name = smart_random_salt_comparison.columns.values.tolist()

['Confidence Interval', 'Number of Groups', 'Number of Days']

In [70]:
df =  smart_random_salt_comparison

In [79]:
new_column_order = column_name[0:3] + [col for col in df.columns if metric_names[0] in col] + [col for col in df.columns if metric_names[1] in col] 

In [82]:
df = df[new_column_order]

In [84]:
df.to_csv('smart_salt_comparison.csv')

In [5]:
account_4_smart_salt_bootstrapping_v3 = pd.DataFrame()
for group_size in ("5percent", "10percent", "25percent"):
    
    table_name = 'account_4_smart_salt_bootstrapping_v3_20190328_{0}_salt_numbers_10'.format(group_size)
    print(table_name)
    smart_salt_bootstrapping = spark.sql("select * from lijia.{table_name}".format(table_name = table_name))\
                .filter("paired_groups == '0_1'")

    num_days   = [7, 14, 21, 28]
    quantile_table = pd.DataFrame()

    for num_day in num_days:

        subdata = smart_salt_bootstrapping.filter(smart_salt_bootstrapping.num_days == num_day).toPandas()
        # manually filter out. 
        #subdata = subdata[np.abs(subdata.rdiff_gross_profit)<5]
        # get the 'true' mean of the underlying distribution, assuming it is the true after resampling many times 
        global_mean = ( subdata.groupby(["num_groups"],as_index=False)
                                                    .agg({'rdiff_' + m:"mean" for m in metric_names}) )
        global_mean.columns = ['num_groups'] + metric_names
        subdata = ( subdata.merge(global_mean,on="num_groups",how="inner") )
        # empirical bootstrap step here: 
        for m in metric_names:
            subdata["eb_"+m] = subdata['rdiff_' + m] - subdata[m] 
        quantile_table = quantile_table.append( subdata.groupby(["num_days", "num_groups"],as_index=False)
                                               .quantile(q = [0.025, 0.975]).round(3)
                                               [['num_groups', "num_days"] + ['eb_' + m for m in metric_names]]
                                              )

    quantile_table.index      = quantile_table.index.set_names('CI', level = 1)
    quantile_table            = quantile_table.reset_index(level = 1)
    quantile_table.CI         = (quantile_table.CI * 100).astype(str) + '%'
    quantile_table.num_days   = quantile_table.num_days.astype(int)
    quantile_table.num_groups = quantile_table.num_groups.astype(int)
    quantile_table.columns = ["Confidence Interval", "Number of Groups", "Number of Days", 
                              "Bootstrapped Relative Difference Smart Salt {bookings}".format(bookings=metric_names[0]),
                              "Bootstrapped Relative Difference Smart Salt {profit}".format(profit=metric_names[1])]

    account_4_smart_salt_bootstrapping_v3 = account_4_smart_salt_bootstrapping_v3.append(quantile_table)


account_4_smart_salt_bootstrapping_v3_20190328_5percent_salt_numbers_10
account_4_smart_salt_bootstrapping_v3_20190328_10percent_salt_numbers_10
account_4_smart_salt_bootstrapping_v3_20190328_25percent_salt_numbers_10


In [7]:
smart_random = random_salt_quantile_table.merge(account_4_smart_salt_bootstrapping_v3, how = 'inner')

In [8]:
smart_random

,Confidence Interval,Number of Groups,Number of Days,Bootstrapped Relative Difference Random Salt nits_bookings,Bootstrapped Relative Difference Random Salt nits_profit,Bootstrapped Relative Difference Smart Salt nits_bookings,Bootstrapped Relative Difference Smart Salt nits_profit
0,2.5%,4,7,-0.034,-0.029,-0.037,-0.022
1,97.5%,4,7,0.035,0.030,0.019,0.034
2,2.5%,10,7,-0.053,-0.046,-0.056,-0.026
3,97.5%,10,7,0.057,0.047,0.059,0.025
4,2.5%,20,7,-0.073,-0.065,-0.031,-0.043
5,97.5%,20,7,0.082,0.065,0.045,0.031
6,2.5%,4,14,-0.030,-0.027,-0.024,-0.017
7,97.5%,4,14,0.031,0.027,0.021,0.028
8,2.5%,10,14,-0.046,-0.043,-0.043,-0.029
9,97.5%,10,14,0.050,0.043,0.049,0.022
